In [ ]:
# Raport z Projektu: Gra Clobber z AI

## Wprowadzenie

Projekt implementuje grę logiczną Clobber wraz z agentami sztucznej inteligencji wykorzystującymi algorytmy Minimax oraz Alfa-Beta do podejmowania decyzji. Celem projektu było stworzenie funkcjonalnej gry, umożliwienie rozgrywki między agentami AI z różnymi heurystykami oraz analiza wydajności zastosowanych algorytmów.

## Struktura Projektu

Projekt składa się z kilku kluczowych modułów:

*   `src/game_state.py`: Definiuje klasę `ClobberGameState` reprezentującą stan gry (plansza, aktualny gracz) oraz logikę ruchów i sprawdzania końca gry.
*   `src/heuristics.py`: Zawiera funkcje heurystyczne oceniające stan gry (mobilność, liczba pionków, izolacja, ocena ogólna).
*   `src/decision_tree.py`: Implementuje klasę `DecisionTree` odpowiedzialną za algorytmy Minimax i Alfa-Beta oraz mechanizm adaptacyjnej zmiany heurystyki.
*   `src/game.py`: Definiuje klasę `ClobberAgent`, która integruje stan gry, heurystyki i drzewo decyzyjne do wyboru ruchu.
*   `main.py`: Prosty skrypt do uruchomienia symulacji gry między dwoma agentami w jednym procesie.
*   `playground.py`: Podobny do `main.py`, służy do eksperymentowania z różnymi konfiguracjami agentów.
*   `player_W.py` / `player_B.py`: Implementują architekturę klient-serwer, gdzie każdy gracz działa jako osobny proces komunikujący się przez gniazda sieciowe.
*   `raport.ipynb`: Niniejszy notatnik Jupyter służący jako raport.

## Kluczowe Komponenty Logiki

### Stan Gry (`ClobberGameState`)

Klasa `ClobberGameState` przechowuje aktualny układ pionków na planszy (reprezentowanej przez tablicę NumPy), informację o tym, który gracz ma ruch, oraz wymiary planszy. Udostępnia metody do:

*   Tworzenia początkowej planszy (`create_clobber_board`).
*   Generowania listy możliwych ruchów dla aktualnego gracza (`get_possible_moves`).
*   Wykonywania ruchu na planszy (`make_move`).
*   Sprawdzania, czy gra się zakończyła (`is_game_over`).
*   Określania zwycięzcy (`check_winner`).

In [ ]:
import numpy as np

class ClobberGameState:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.board = self.create_clobber_board(rows, cols)
        self.current_player = 'W'

    def create_clobber_board(self, rows, cols):
        board = np.empty((rows, cols), dtype=str)
        for r in range(rows):
            for c in range(cols):
                board[r, c] = 'W' if (r + c) % 2 == 0 else 'B'
        return board
    
    def get_possible_moves(self, print_moves=False):
        """
        Get all possible moves for the current player.
        """
        moves = []
        for r in range(self.rows):
            for c in range(self.cols):
                if self.board[r, c] == self.current_player:
                    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        new_r, new_c = r + dr, c + dc
                        if 0 <= new_r < self.rows and 0 <= new_c < self.cols:
                            if self.board[new_r, new_c] != self.current_player and self.board[new_r, new_c] != '_':
                                moves.append(((r, c), (new_r, new_c)))
                                if print_moves:
                                    print("Possible move:", self.board[r, c], "to", self.board[new_r, new_c])
        return moves
    
    def make_move(self, move):
        """
        Make a move on the board.
        """
        (start_r, start_c), (end_r, end_c) = move
        if self.board[start_r, start_c] != self.current_player:
            raise ValueError("Invalid move: not the current player's piece")
        if self.board[end_r, end_c] == self.current_player:
            raise ValueError("Invalid move: cannot move to the same color")
        self.board[end_r, end_c] = self.current_player
        self.board[start_r, start_c] = '_'
        self.current_player = 'B' if self.current_player == 'W' else 'W'
        return self
    
    def is_game_over(self):
        """
        Check if the game is over.
        """
        return self.get_possible_moves() == []
    
    def check_winner(self):
        if self.is_game_over():
            if self.current_player == 'W':
                return 'B'
            else:
                return 'W'
        return None
    
    def get_num_of_pieces(self, player):
        """
        Get the number of pieces for a player.
        """
        return np.sum(self.board == player)


### Heurystyki (`heuristics.py`)

Moduł ten dostarcza funkcji oceniających jakość danego stanu gry z perspektywy konkretnego gracza. Zaimplementowane heurystyki to:

*   `evaluate`: Kombinacja różnych czynników (mobilność, liczba pionków, izolacja) z wagami.
*   `mobility_score`: Różnica w liczbie możliwych ruchów między graczem a przeciwnikiem.
*   `piece_count_score`: Różnica w liczbie pionków na planszy.
*   `isolation_score`: Różnica w liczbie izolowanych pionków przeciwnika i gracza (izolowany pionek nie ma sąsiadujących pionków żadnego koloru).

In [ ]:

from game_state import ClobberGameState


def evaluate(game_state : ClobberGameState, player):
    opponent = 'B' if player == 'W' else 'W'
    
    def is_isolated(x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(game_state.board) and 0 <= ny < len(game_state.board[0]):
                if game_state.board[nx][ny] in ('B', 'W'):
                    return False
        return True

    my_pieces = opp_pieces = my_moves = opp_moves = my_isolated = opp_isolated = 0
    for x in range(len(game_state.board)):
        for y in range(len(game_state.board[0])):
            piece = game_state.board[x][y]
            if piece == player:
                my_pieces += 1
                if is_isolated(x, y): my_isolated += 1
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < len(game_state.board) and 0 <= ny < len(game_state.board[0]):
                        if game_state.board[nx][ny] == opponent:
                            my_moves += 1
            elif piece == opponent:
                opp_pieces += 1
                if is_isolated(x, y): opp_isolated += 1
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < len(game_state.board) and 0 <= ny < len(game_state.board[0]):
                        if game_state.board[nx][ny] == player:
                            opp_moves += 1

    score = (
        10 * (my_moves - opp_moves) +
        20 * (my_pieces - opp_pieces) +
        15 * (opp_isolated - my_isolated)
    )
    return score

def mobility_score(game_state: ClobberGameState, player):
    opponent = 'B' if player == 'W' else 'W'
    def count_moves(p):
        moves = 0
        for x in range(len(game_state.board)):
            for y in range(len(game_state.board[0])):
                if game_state.board[x][y] == p:
                    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < len(game_state.board) and 0 <= ny < len(game_state.board[0]):
                            if game_state.board[nx][ny] == ('B' if p == 'W' else 'W'):
                                moves += 1
        return moves
    return count_moves(player) - count_moves(opponent)

def piece_count_score(game_state: ClobberGameState, player):
    opponent = 'B' if player == 'W' else 'W'
    my_count= 0
    opp_count= 0
    for x in range(len(game_state.board)):
        for y in range(len(game_state.board[0])):
            piece = game_state.board[x][y]
            if piece == player:
                my_count += 1
            elif piece == opponent:
                opp_count += 1
    return my_count - opp_count

def isolation_score(game_state: ClobberGameState, player):
    opponent = 'B' if player == 'W' else 'W'
    
    def is_isolated(x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(game_state.board) and 0 <= ny < len(game_state.board[0]):
                if game_state.board[nx][ny] in ('B', 'W'):
                    return False
        return True

    def count_isolated(p):
        count = 0
        for x in range(len(game_state.board)):
            for y in range(len(game_state.board[0])):
                if game_state.board[x][y] == p and is_isolated(x, y):
                    count += 1
        return count

    return count_isolated(opponent) - count_isolated(player)


def heuristic_evaluate(game_state: ClobberGameState):
    """
    Evaluate the game state for the given player.
    """
    board = game_state.board
    player = game_state.current_player
    return evaluate(board, player)


### Drzewo Decyzyjne i Algorytmy (`DecisionTree`)

Klasa `DecisionTree` jest sercem procesu decyzyjnego agentów AI. Implementuje:

*   **Algorytm Minimax**: Przeszukuje drzewo gry, minimalizując maksymalną możliwą stratę.
*   **Algorytm Alfa-Beta**: Optymalizacja algorytmu Minimax poprzez odcinanie gałęzi drzewa, które na pewno nie wpłyną na ostateczny wynik.
*   **Ograniczenie Głębokości**: Przeszukiwanie jest ograniczone do zadanej głębokości (`max_depth`), aby zapewnić rozsądny czas obliczeń.
*   **Buforowanie Heurystyki (Cache)**: Przechowuje wyniki obliczeń heurystyki dla już odwiedzonych stanów, aby uniknąć powtórnych obliczeń.
*   **Adaptacyjna Zmiana Heurystyki**: (Opcjonalnie) Analizuje stan gry (np. etap gry - początkowy, środkowy, końcowy) i dynamicznie wybiera najbardziej odpowiednią heurystykę w danym momencie.

In [ ]:

import copy
from game_state import ClobberGameState
from heuristics import mobility_score, piece_count_score, isolation_score

class DecisionTree:
    def __init__(self, max_depth, game_state: ClobberGameState, heuristic, strategy='minmax', player=None):
        """
        Initialize the decision tree with the game state and strategy.
        """
        self.max_depth = max_depth
        self.tree = self.crate_tree(game_state)
        self.strategy = strategy
        self.heuristic = heuristic
        self.player = player
        self.num_of_visits = 0
        self.heuristic_cache = {}  

    def crate_tree(self, game_state):
        """
        Create a decision tree from the given game state.
        """
        self.tree = self._build_tree(game_state, depth=0)
        return self.tree

    def _build_tree(self, game_state: ClobberGameState, depth):
        """
        Recursively build the decision tree.
        """
        if self.max_depth is not None and depth >= self.max_depth:
            return None

        if game_state.is_game_over():
            return game_state.check_winner()

        possible_moves = game_state.get_possible_moves(print_moves=False)

        tree = {}
        for move in possible_moves:
            copied_game_state = copy.deepcopy(game_state)
            new_game_state = copied_game_state.make_move(move)
            tree[move] = self._build_tree(copy.deepcopy(new_game_state), depth + 1)

        return tree
    
    def get_best_move(self, game_state: ClobberGameState):
        """
        Get the best move for the current player using the heuristic.
        """
        best_move_so_far = None
        if self.strategy == 'minmax':
            best_move_so_far = (None, float('-inf'))
            for move in game_state.get_possible_moves():
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                move_value = self.minimax_search(temp_game_state, self.max_depth - 1, False)
                
                if move_value > best_move_so_far[1]:
                    best_move_so_far = (move, move_value)
        elif self.strategy == 'alpha-beta':
            best_move_so_far = (None, float('-inf'))
            for move in game_state.get_possible_moves():
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                move_value = self.alfa_beta_search(temp_game_state, self.max_depth - 1, float('-inf'), float('inf'), False)
                
                if move_value > best_move_so_far[1]:
                    best_move_so_far = (move, move_value)
        return best_move_so_far[0] if best_move_so_far else None
    
    def minimax_search(self, game_state: ClobberGameState, depth, maximizing_player):
        """
        Perform a minimax search on the game state.
        game_state: ClobberGameState
        depth: Depth of the search
        maximizing_player: Boolean indicating if it's the maximizing player's turn        
        """
        state_key = (str(game_state.board), depth, maximizing_player)
        if state_key in self.heuristic_cache:
            return self.heuristic_cache[state_key]
            
        if depth == 0 or game_state.is_game_over():
            result = self.heuristic(game_state, self.player)
            self.heuristic_cache[state_key] = result
            return result

        possible_moves = game_state.get_possible_moves()
        if not possible_moves:  
            result = self.heuristic(game_state, self.player)
            self.heuristic_cache[state_key] = result
            return result

        if maximizing_player:
            max_eval = float('-inf')
            for move in possible_moves:
                self.num_of_visits += 1
                
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                eval_value = self.minimax_search(temp_game_state, depth - 1, False)
                max_eval = max(max_eval, eval_value)
            
            self.heuristic_cache[state_key] = max_eval
            return max_eval
        else:
            min_eval = float('inf')
            for move in possible_moves:
                self.num_of_visits += 1
                
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                eval_value = self.minimax_search(temp_game_state, depth - 1, True)
                min_eval = min(min_eval, eval_value)
            
            self.heuristic_cache[state_key] = min_eval
            return min_eval
    
    def alfa_beta_search(self, game_state: ClobberGameState, depth, alpha, beta, maximizing_player):
        """
        Perform an alpha-beta search on the game state.
        """
        state_key = (str(game_state.board), depth, maximizing_player)
        if state_key in self.heuristic_cache:
            return self.heuristic_cache[state_key]
            
        if depth == 0 or game_state.is_game_over():
            result = self.heuristic(game_state, self.player)
            self.heuristic_cache[state_key] = result
            return result

        possible_moves = game_state.get_possible_moves(print_moves=False)
        if not possible_moves:  
            result = self.heuristic(game_state, self.player)
            self.heuristic_cache[state_key] = result
            return result

        if maximizing_player:
            max_eval = float('-inf')
            for move in possible_moves:
                self.num_of_visits += 1
                
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                eval_value = self.alfa_beta_search(temp_game_state, depth - 1, alpha, beta, False)
                
                max_eval = max(max_eval, eval_value)
                alpha = max(alpha, eval_value)
                if beta <= alpha:
                    break
            
            self.heuristic_cache[state_key] = max_eval
            return max_eval
        else:
            min_eval = float('inf')
            for move in possible_moves:
                self.num_of_visits += 1
                
                temp_game_state = copy.deepcopy(game_state)
                temp_game_state.make_move(move)
                
                eval_value = self.alfa_beta_search(temp_game_state, depth - 1, alpha, beta, True)
                
                min_eval = min(min_eval, eval_value)
                beta = min(beta, eval_value)
                if beta <= alpha:
                    break
            
            self.heuristic_cache[state_key] = min_eval
            return min_eval
    
    def analyze_and_change_heuristic(self, game_state: ClobberGameState):
        """
        Analyze the game state and change the heuristic based on the analysis.
        Choose the most appropriate heuristic based on the current game state.
        """
        mobility = mobility_score(game_state, self.player)
        piece_count = piece_count_score(game_state, self.player)
        isolation = isolation_score(game_state, self.player)
        
        total_pieces = game_state.get_num_of_pieces('W') + game_state.get_num_of_pieces('B')
        max_pieces = game_state.cols * game_state.rows
        game_progress = 1 - (total_pieces / max_pieces) 
        
        epsilon = 1e-10
        scores = {
            "mobility": abs(mobility),
            "piece_count": abs(piece_count),
            "isolation": abs(isolation)
        }
        
        max_score = max(scores.values()) + epsilon
        normalized_scores = {k: v/max_score for k, v in scores.items()}
        
        if game_progress < 0.3:  # Erly game
            weights = {"mobility": 0.7, "piece_count": 0.2, "isolation": 0.1}
        elif game_progress < 0.7:  # midgame
            weights = {"mobility": 0.4, "piece_count": 0.4, "isolation": 0.2}
        else:  # Lategame
            weights = {"mobility": 0.2, "piece_count": 0.3, "isolation": 0.5}
        
        weighted_scores = {
            "mobility": normalized_scores["mobility"] * weights["mobility"],
            "piece_count": normalized_scores["piece_count"] * weights["piece_count"],
            "isolation": normalized_scores["isolation"] * weights["isolation"]
        }
        
        best_heuristic_name = max(weighted_scores, key=weighted_scores.get)
        
        heuristic_map = {
            "mobility": mobility_score,
            "piece_count": piece_count_score,
            "isolation": isolation_score
        }
        
        new_heuristic = heuristic_map[best_heuristic_name]
        
        if self.heuristic.__code__ != new_heuristic.__code__:
            old_heuristic_name = next((name for name, func in heuristic_map.items() 
                                      if func.__code__ == self.heuristic.__code__), "unknown")
            
            print(f"Game progress: {game_progress:.2f} (Phase: {'early' if game_progress < 0.3 else 'mid' if game_progress < 0.7 else 'late'})")
            print(f"Raw scores: Mobility={mobility:.2f}, Piece Count={piece_count:.2f}, Isolation={isolation:.2f}")
            print(f"Normalized scores: {', '.join([f'{k}={v:.2f}' for k, v in normalized_scores.items()])}")
            print(f"Weighted scores: {', '.join([f'{k}={v:.2f}' for k, v in weighted_scores.items()])}")
            print(f"Changing heuristic from {old_heuristic_name} to {best_heuristic_name}")
            
            return new_heuristic
        return None

### Agent Gry (`ClobberAgent`)

Klasa `ClobberAgent` łączy wszystkie komponenty. Inicjalizowana jest z określoną nazwą (np. 'W' lub 'B'), początkowym stanem gry, wybraną heurystyką, strategią (Minimax lub Alfa-Beta) i maksymalną głębokością przeszukiwania. Metoda `play` tej klasy:

1.  Tworzy instancję `DecisionTree`.
2.  (Opcjonalnie) Wywołuje mechanizm adaptacyjnej zmiany heurystyki.
3.  Używa drzewa decyzyjnego do znalezienia najlepszego ruchu (`get_best_move`).
4.  Wykonuje znaleziony ruch na obiekcie stanu gry.
5.  Zwraca zaktualizowany stan gry lub `None`, jeśli gra się zakończyła.

In [ ]:
import copy
import sys
from game_state import ClobberGameState
from decision_tree import DecisionTree
class ClobberAgent:
    def __init__(self, name, initial_game_state, heuristic, strategy='minmax',max_depth=None, adaptive=False):
        self.name = name
        self.game_state = initial_game_state
        self.heuristic = heuristic
        self.strategy = strategy
        self.max_depth = max_depth
        self.adaptive = adaptive

    def play(self, game: ClobberGameState):
        """
        Play a move using the decision tree strategy.
        """
        print(f"{self.name} is playing...")
        print(f"Current board:\n{game.board}")
        print(f"Current player: {game.current_player}")
        print(f"Evaluating moves using {self.strategy} strategy...")

        dt=DecisionTree(self.max_depth, copy.deepcopy(game), self.heuristic, self.strategy, self.name)

        if self.adaptive:
            potential_new_heuristic = dt.analyze_and_change_heuristic(copy.deepcopy(game))
            self.heuristic= potential_new_heuristic if potential_new_heuristic else self.heuristic
        best_move = dt.get_best_move(copy.deepcopy(game))
        print(f"Number of nodes visited: {dt.num_of_visits}", file=sys.stderr)
        if best_move:
            game.make_move(best_move)
            print(f"{self.name} played move {best_move}")
        else:
            print(f"{self.name} has no valid moves. Game over.")
        winner = game.check_winner()
        if winner:
            print(f"Winner: {winner}")
            print(f"Final board:\n{game.board}")
            return None
        else:
            print("No winner yet.")
        return game





## Użyte Biblioteki

*   **NumPy**: Do efektywnej reprezentacji i manipulacji planszą gry.
*   **Copy**: Do tworzenia głębokich kopii obiektów stanu gry, co jest kluczowe w algorytmach przeszukiwania drzewa.
*   **Multiprocessing**: (W `player_W.py` i `player_B.py`) Do komunikacji między procesami agentów za pomocą `Listener` i `Client`.
*   **Sys**: Do zarządzania ścieżkami modułów i przekierowywania danych wyjściowych (np. liczby odwiedzonych węzłów na stderr).
*   **Datetime**: Do mierzenia czasu wykonania algorytmów.

## Uruchamianie Gry

Grę można uruchomić na kilka sposobów:

1.  **Symulacja w jednym procesie**: Uruchomienie `main.py` lub `playground.py`. Obaj agenci działają sekwencyjnie w tym samym procesie.
2.  **Rozgrywka klient-serwer**: Należy uruchomić `player_B.py` (serwer/nasłuchujący), a następnie `player_W.py` (klient/łączący się). Każdy agent działa w osobnym procesie, komunikując się przez gniazda lokalne.

In [ ]:
import sys
sys.path.append("src")
from src.game import ClobberAgent
from src.game_state import ClobberGameState
from src.heuristics import evaluate, mobility_score, piece_count_score, isolation_score

def simulate_game(heuristic_W, heuristic_B, strategy_A, strategy_B, max_depth_A, max_depth_B, rows, cols):
    """
    Simulate a game of Clobber between two agents.
    """
    initial_game_state = ClobberGameState(rows,cols)
    agent = ClobberAgent(name="W", initial_game_state=initial_game_state, heuristic=heuristic_W, strategy=strategy_A, max_depth=max_depth_A)
    agent2 = ClobberAgent(name="B", initial_game_state=initial_game_state, heuristic=heuristic_B, strategy=strategy_B, max_depth=max_depth_B)
    num_of_moves = 1
    print("Starting game...")
    move=agent.play(initial_game_state)
    while move:
        num_of_moves += 1
        move=agent2.play(move)
        if move:
            move=agent.play(move)
        else:
            break
    print("Game Over")
    print(f"Total moves played: {num_of_moves}")


if __name__ == "__main__":
    rows = 8
    cols = 8
    heuristic_W = isolation_score
    heuristic_B = mobility_score
    # alpha-beta/minmax
    strategy_A = 'alpha-beta'
    strategy_B = 'alpha-beta'
    max_depth_A = 10
    max_depth_B = 10

    simulate_game(heuristic_W, heuristic_B, strategy_A, strategy_B, max_depth_A, max_depth_B, rows, cols)

## Wnioski i Możliwe Rozszerzenia

Projekt skutecznie implementuje grę Clobber z agentami AI wykorzystującymi Minimax i Alfa-Beta. Kluczowe cechy to modularna struktura, różne heurystyki i możliwość adaptacyjnej zmiany strategii.

Możliwe rozszerzenia:
*   Implementacja bardziej zaawansowanych heurystyk.
*   Dodanie graficznego interfejsu użytkownika (GUI).
*   Optymalizacja algorytmów (np. Iterative Deepening Depth First Search - IDDFS).